In [10]:
import pandas as pd
import geopandas as gpd
import folium
from shapely.geometry import LineString
from IPython.display import IFrame
import os

# Asegura que la carpeta output exista
os.makedirs("../output", exist_ok=True)

# Cargar archivos
df_result = pd.read_csv("../output/resultados_validacion.csv")
df_pois = pd.read_csv("../data/POIs/POI_4815075.csv")
gdf_calles = gpd.read_file("../data/STREETS_NAV/SREETS_NAV_4815075.geojson")

# Normalizar columnas
df_result.columns = [col.upper() for col in df_result.columns]
df_pois.columns = [col.upper() for col in df_pois.columns]
gdf_calles.columns = [col.lower() for col in gdf_calles.columns]

# Unir resultados con POIs para recuperar info necesaria
df = pd.merge(df_result, df_pois, on="POI_ID", how="left")

# Diccionario de colores por acción
color_dict = {
    "eliminar": "red",
    "corregir lado (POI_ST_SD)": "orange",
    "cambiar MULTIDIGIT a N": "blue",
    "marcar como excepción": "green"
}

# Lista para guardar puntos
points = []

# Recorrer POIs y reconstruir coordenadas
for _, row in df.iterrows():
    try:
        link_id = row["LINK_ID"]
        perc = float(row["PERCFRREF"])

        calle_match = gdf_calles[gdf_calles["link_id"] == link_id]

        if calle_match.empty:
            continue

        geom = calle_match.iloc[0].geometry
        if not isinstance(geom, LineString):
            continue

        point = geom.interpolate(perc * geom.length)
        lat, lon = point.y, point.x

        points.append({
            "POI_ID": row["POI_ID"],
            "LAT": lat,
            "LON": lon,
            "ACCION": row["ACCION"],
            "COMENTARIO": row["COMENTARIO"]
        })

    except Exception as e:
        continue

# Crear DataFrame con coordenadas
df_coords = pd.DataFrame(points)

# Crear mapa
mapa = folium.Map()
bounds = [[df_coords["LAT"].min(), df_coords["LON"].min()],
          [df_coords["LAT"].max(), df_coords["LON"].max()]]
mapa.fit_bounds(bounds)

# Agregar puntos al mapa
for _, row in df_coords.iterrows():
    color = color_dict.get(row["ACCION"], "gray")
    popup = f"<b>{row['ACCION']}</b><br>{row['COMENTARIO']}"
    folium.CircleMarker(
        location=[row["LAT"], row["LON"]],
        radius=4,
        color=color,
        fill=True,
        fill_opacity=0.75,
        popup=folium.Popup(popup, max_width=300)
    ).add_to(mapa)

# Agregar leyenda
legend_html = """
<div style='position: fixed; bottom: 50px; left: 50px; width: 230px; height: 120px;
     border:2px solid grey; z-index:9999; font-size:14px; background-color:white;
     padding: 10px; box-shadow: 2px 2px 5px rgba(0,0,0,0.3);'>
<b>🗺️ Leyenda:</b><br>
🔴 Eliminar<br>
🟠 Corregir lado<br>
🔵 Cambiar MULTIDIGIT<br>
🟢 Excepción válida
</div>
"""
mapa.get_root().html.add_child(folium.Element(legend_html))

# Guardar mapa y mostrarlo
mapa.save("../output/mapa_resultado.html")
IFrame("../output/mapa_resultado.html", width=900, height=500)
